In [2]:
import os
import sys
import glob
import numpy as np
from skimage import io
from sklearn import datasets
from skimage.feature import hog

CATEGORY_NUM = 6

## ラベル名(0～)を付けたディレクトリに分類されたイメージファイルを読み込む
## 入力パスはラベル名の上位のディレクトリ
def load_handimage(path):

    # ファイル一覧を取得
    files = glob.glob(os.path.join(path, '*/*.png'))

    # イメージとラベル領域を確保
    hogs = np.ndarray((len(files), 3600), dtype = np.float)
    labels = np.ndarray(len(files), dtype=np.int)

    # イメージとラベルを読み込み
    for idx, file in enumerate(files):
        # イメージ読み込み
        image = io.imread(file, as_gray=True)
        h = hog(image, orientations=9, pixels_per_cell=(5, 5),
            cells_per_block=(5, 5))
        hogs[idx] = h

        # ディレクトリ名よりラベルを取得
        label = os.path.split(os.path.dirname(file))[-1]
        labels[idx] = int(label)

    return datasets.base.Bunch(data=hogs,
                 target=labels.astype(np.int),
                 target_names=np.arange(CATEGORY_NUM),
                 DESCR=None)

#####################################
from sklearn import svm, metrics
from sklearn import model_selection

param_grid = {
   'C': [1, 10, 100],
   'loss': ['hinge', 'squared_hinge']
   }
   
## usage:
##    python classify_handsign_1.py <n> <dir_1> <dir_2> ... <dir_m>
##      n          テスト用データディレクトリ数
##      dir_1      データディレクトリ1
##      dir_m      データディレクトリm

if __name__ == '__main__':
    prepath="./learning_sample/06/data/"
    # 評価用ディレクトリ数の取得
    paths_for_test=["m01","m05","m06","m07"]
    paths_for_train=["m02","m03","m04c","m08","m09","m10","m11","m12","m13","m14","m15","m16"]
    
    print('test ', paths_for_test)
    print('train', paths_for_train)

    # 学習データの読み込み
    data = []
    label = []
    for i in range(len(paths_for_train)):
        path = paths_for_train[i]
        path=os.path.join(prepath,path)
        d = load_handimage(path)
        data.append(d.data)
        label.append(d.target)
    train_data = np.concatenate(data)
    train_label = np.concatenate(label)

    # 手法:線形SVM
    classifier = model_selection.GridSearchCV(svm.LinearSVC(), param_grid,n_jobs=-1)
    
    # 学習
    classifier.fit(train_data, train_label)

    # Grid Search結果表示
    print("Best Estimator:\n%s\n"%classifier.best_estimator_)
    for params, mean_score, all_scores in classifier.grid_scores_:
        print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, 
            all_scores.std() * 2, params))
            
    for path in paths_for_test:
        # テストデータの読み込み
        path=os.path.join(prepath,path)
        d = load_handimage(path)
        
        # テスト
        predicted = classifier.predict(d.data)

        # 結果表示
        print("### %s ###" % path)
        print("Accuracy:\n%s"
            % metrics.accuracy_score(d.target, predicted))
        print("Classification report:\n%s\n"
            % metrics.classification_report(d.target, predicted))


test  ['m01', 'm05', 'm06', 'm07']
train ['m02', 'm03', 'm04c', 'm08', 'm09', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15', 'm16']
Best Estimator:
LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

0.764 (+/- 0.002) for {'C': 1, 'loss': 'hinge'}
0.769 (+/- 0.002) for {'C': 1, 'loss': 'squared_hinge'}
0.769 (+/- 0.002) for {'C': 10, 'loss': 'hinge'}
0.778 (+/- 0.002) for {'C': 10, 'loss': 'squared_hinge'}
0.771 (+/- 0.003) for {'C': 100, 'loss': 'hinge'}
0.774 (+/- 0.004) for {'C': 100, 'loss': 'squared_hinge'}


C:\Users\mgt\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


### ./learning_sample/06/data/m01 ###
Accuracy:
0.8533333333333334
Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       100
          1       1.00      0.90      0.95       100
          2       0.71      1.00      0.83       100
          3       0.81      0.29      0.43       100
          4       0.69      0.93      0.79       100
          5       1.00      1.00      1.00       100

avg / total       0.87      0.85      0.83       600


### ./learning_sample/06/data/m05 ###
Accuracy:
0.87
Classification report:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99       100
          1       0.95      1.00      0.98       100
          2       0.79      0.82      0.80       100
          3       0.78      0.49      0.60       100
          4       0.74      0.91      0.82       100
          5       0.97      1.00      0.99       100

avg / total       0.87      0.87 